# LionAGI introduction 3 - LLM sessions

In [1]:
import lionagi as li

In [2]:
system = """
You are a helpful assistant. You are asked to perform as a calculator. Return as an integer.
"""

calculator = li.Session(system=system)

In [3]:
a = -3
b = 4

context = {
    "number1": a,
    "number2": b,
}

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values",}

instruct2 = {
    "multiplication": "provided with 2 numbers, return their multiplication",}

instruct3 = {
    "case positive": "if the result from previous step is positive, times 2 to the previous step's result",
    "case negative": "elif the result from previous step is negative, plus 2 to the previous step's result",
    "case zero": "elif the result from previous step is zero, return the previous step's result",}


In [4]:
await calculator.initiate(instruction=instruct1, context=context, model="gpt-3.5-turbo")

'7'

In [5]:
cal1 = await calculator.initiate(instruction=instruct1, context=context, model="gpt-3.5-turbo")
cal2 = await calculator.followup(instruction=instruct3, model="gpt-4", temperature=0.5)

print(f"Given {a} and {b}, the sum of absolute values is {cal1}")
print(f"Since the step 1 result is {'positive' if int(cal1)>0 else 'negative'}, the second step result is {cal2}")

Given -3 and 4, the sum of absolute values is 7
Since the step 1 result is positive, the second step result is 14


In [6]:
cal1 = await calculator.initiate(instruction=instruct2, context=context, model="gpt-3.5-turbo")
cal2 = await calculator.followup(instruction=instruct3, model="gpt-4", temperature=0.5, n=3)

print(f"Given {a} and {b}, the multiplication product is {cal1}")
print(f"Since the step 1 result is {'positive' if int(cal1)>0 else 'negative'}, the second step result is {cal2}")

Given -3 and 4, the multiplication product is -12
Since the step 1 result is negative, the second step result is -10


In [7]:
# since the result from a llm is not always reliable, 
# we can add another layer of llm trying to validate it to reduce eror rate

intruct4 = {"string to number": "return as a number by itself",}

cal4 = await calculator.followup(instruction=intruct4, model="gpt-4", temperature=0.5, n=3)


In [8]:
cal4

'-10'

In [9]:
a = 3
b = 4

context = {
    "number1": a,
    "number2": b,
}

await calculator.initiate(instruction=instruct2, context=context, model="gpt-3.5-turbo")

'12'

In [10]:
await calculator.followup(instruction=instruct3, model="gpt-3.5-turbo")

'24'

In [11]:
#### ok now let's see how we can make it more interesting
import numpy as np

ints1 = np.random.randint(-10, 10, size=5)
ints2 = np.random.randint(0, 10, size=5)
cases = np.random.randint(0,2, size=5)
# let's define a simple parser function

f = lambda i: {"number1": str(ints1[i]), "number2": str(ints2[i]), "case_": str(cases[i])}
contexts = li.l_return(range(5), f)

In [12]:
system = """
You are a helpful assistant. You are asked to perform as a calculator. Return as an integer.
"""

context = {
    "number1": a,
    "number2": b,
}

instruct1 = {
    "sum the absolute values": "provided with 2 numbers, return the sum of their absolute values. i.e. |x|+|y|",}

instruct2 = {
    "diff the absolute values": "provided with 2 numbers, return the difference of absolute values. i.e. |x|-|y|",}

instruct3 = {
    "if previous response is positive": "times 2. i.e. *2", # case 1
    "else": "plus 2. i.e. +2",                              # case 2
}

In [19]:
async def calculator_workflow(context_):
    calculator = li.Session(system=system)
    context = context_.copy()
    case = int(context.pop("case_"))
    
    if case == 0:
        await calculator.initiate(instruction=instruct1, context=context, model="gpt-4", temperature=0.5)
    elif case == 1:
        await calculator.initiate(instruction=instruct2, context=context, model="gpt-4", temperature=0.5)
    
    await calculator.followup(instruction=instruct3, model="gpt-4", temperature=0.3)
    return li.l_return(calculator.conversation.responses, lambda i: i['content'])

In [20]:
outs = await li.async_l_return(contexts, calculator_workflow)

In [21]:
for idx, out in enumerate(outs):
    print(f"Inputs: {ints1[idx]}, {ints2[idx]}, case: {cases[idx]}")
    print(f"Outputs: {out}")
    print("------")

Inputs: -9, 0, case: 1
Outputs: ['9', '18']
------
Inputs: -2, 0, case: 0
Outputs: ['2', '4']
------
Inputs: -1, 5, case: 0
Outputs: ['6', '12']
------
Inputs: 7, 8, case: 1
Outputs: ['-1', '1']
------
Inputs: 0, 6, case: 1
Outputs: ['-6', '-4']
------
